# Clustering Neighborhoods in Toronto
### IBM Data Science Capstone - Week 3

**Goals: Explore the neighborhoods of Toronto and group them by the most common venues they contain. Visualize this information.**

First, we need to setup the notebook with the packages we will need to complete all of the tasks.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy # uncomment this line if you need to isntall geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!pip install geocoder # uncomment this line if you need to install geocoder
import geocoder
import requests

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !pip install folium # uncomment this line if you need to install folium
import folium # map rendering library

import urllib.request, urllib.parse, urllib.error # library to handle url parsing for webscraping

#!pip install beautifulsoup4 # uncomment this line if you need to install beautifulsoup
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


Next, we will need to be able to define the neighborhoods in Toronto. This will be done using postal codes. We can find a list of all [postal codes for Ontario (the providence that Toronto is in) on Wikipedia.](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

Using this, we can build a *pandas* dataframe that contains the postal codes of Toronoto's neighborhoods. To create the above dataframe:

* The dataframe will consist of three columns: **PostalCode, Borough, and Neighborhood**
* Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.
* If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.
* In the last cell of the notebook, we will use the .shape method to print the number of rows of the *pandas* dataframe.

To begin, we need to scrape the Wikipedia page for the information we need. To do this, we will used parts of *urllib* and *BeautifulSoup*. *urllib* allows us to easily acquire the HTML associated with a page, and *BeautifulSoup* lets us make sense of that information, as HTML is often quite messy when you scrape it. We will be looking for the ```<table>``` tags to indicate the table on the page, and then work through the lines of the table to build a *pandas* dataframe of the information we need.

Let's do our setup for this step.

In [2]:
data = [] # empty list for storing information

# Request a local copy of the Article

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('Toronto_Postal_codes.html', 'w') as fo:
    fo.write(article)

Now that we have a local copy of the article, we can 'soupify' it and parse the HTML into something we can iterate through.

In [3]:
# Load article, turn into soup and get the <table>s.
article = open('Toronto_Postal_codes.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postal Code', 'Borough', 'Neighbourhood']:
        break

Now that we have parsed the HTML into something we can iterate, we can work through it and find the information we need. We will store that in a list of dictionaries, as that is easier to append than a dataframe.

In [4]:
# Extract the columns we want and write to list of dictionaries.
with open('Toronto_Postal_codes.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        pc, b, n = [td.text.strip() for td in tds[:3]]
        
        # filter to ignore bouroughs that are 'Not Assigned'
        if b != 'Not assigned':
            # filter to replace 'Not assigned' neighborhoods with bourough names
            if n == 'Not assigned':
                data.append({'Postal Code':pc,'Borough':b,'Neighbourhood':b})
            else: 
                data.append({'Postal Code':pc,'Borough':b,'Neighbourhood':n})

Now that we have all the information we want (and none of the information we don't) we can make the dataframe, and set the index to be the postal code.

In [5]:
# Convert the list of Dictionaries into a padas dataframe
df_postal = pd.DataFrame(data)
df_postal

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
df_postal.shape

(103, 3)

**This completes Section 1.**

#### Get the coordinates for all neighborhoods in Toronto.

The next step in our process is to find and add the geolocation data (latitude and logitude) for each postal code in our dataset. To do this, we will use a known set of coordinates for each postal code. This information is stored in a CSV, so we will read that into a dataframe.

In [7]:
# intialize lists for data storage
lat = []
lng = []

# initialize the file
df_latlng = pd.read_csv('https://cocl.us/Geospatial_data')
df_latlng

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Now we will merge the two dataframes using ```how='inner'``` and ```on='Postal Code'``` so that we don't create any new rows.

In [8]:
df_merge = df_postal.merge(df_latlng, how='inner', on='Postal Code')
df_merge

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


**This Completes Section 2.**

Now that we have the data gathered, cleaned, and organized, we can start workin on analysis. We want to explore and cluster the neighborhoods of Toronto. We will aprooach this problem by:

* Visulalize the existing data.
* Exploring each neighborhood using Foursquare's API
* Analyzing the neighborhood using one hot encoding and commonality
* Using k-means clustering to cluster the neighborhoods
* Explore and visualize the clusters

#### Use the Geopy library to get the latitude and lofitude of Toronto.

In order to define an instance of the geocoder, we need to define a user agent. We are going to call our toronto_expolerer.

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Build a map of Toronto and all the neighborhoods it contains.

In [10]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Use the FourSquare API to explore the neighborhoods in Toronto

Now we can use the API from FourSquare to start to learn mroe about the neighborhoods in Toronto. Specifically, we want to know what the most popular venues are in each neighborhood, so we that can build clusters from that. I've added a hidden frame with my FourSquare credentials, so you will need to add your own to the code in order to replicate.

In [11]:
# The code was removed by Watson Studio for sharing.

Let's start by exploring the first neighborhood in our dataframe. We will get the name of the neighborhood and then return that along with it's coordinates.

In [12]:
neighborhood_latitude = df_merge.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_merge.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_merge.loc[0, 'Neighbourhood'].split(',') # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Parkwoods'] are 43.7532586, -79.3296565.


With this information, we can request the top 100 venues within 500 meters of Parkwoods from FourSquare. We will save that information in a vvariable called ```results```.

In [23]:
radius = 1000
LIMIT = 100

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

Based on prior knowledge and documentation, we know that what we want to pull out of the results is stored in the *items* key. We also know we need to know the venue category type. Let's work on those issues now.

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Brookbanks Park,Park,43.751976,-79.332140
2,Tim Hortons,Café,43.760668,-79.326368
3,Bruno's valu-mart,Grocery Store,43.746143,-79.324630
4,A&W,Fast Food Restaurant,43.760643,-79.326865


How many venues did we find for Parkwoods?

In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

28 venues were returned by Foursquare.


#### Expand to all of Toronto

We can use a function to repeat this process for every neighborhood in our data frame.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, we can run the function and build the nearby venues data frame.

In [20]:
toronto_venues = getNearbyVenues(names=df_merge['Neighbourhood'],
                                latitudes=df_merge['Latitude'],
                                longitudes=df_merge['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(4878, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
4,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant


Let's see how many venues we found in each neighborhood.

In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,50,50,50,50,50,50
"Alderwood, Long Branch",24,24,24,24,24,24
"Bathurst Manor, Wilson Heights, Downsview North",29,29,29,29,29,29
Bayview Village,14,14,14,14,14,14
"Bedford Park, Lawrence Manor East",41,41,41,41,41,41
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",15,15,15,15,15,15
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",50,50,50,50,50,50


In [28]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 334 unique categories.


#### Let's analyze each neighborhood and normalize the data.

We will start with one hoe encoding, or making each type of venue a binary value in its own column.

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
first_col = ['Neighborhood']
fixed_columns = [col for col in toronto_onehot if col not in first_col]
toronto_onehot = toronto_onehot[first_col + fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack 

In [30]:
toronto_onehot.shape

(4878, 334)

Now, we can group the rows by neighborhood and take the frequency mean of each venue category.

In [31]:
toronto_groups = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_groups.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack 

In [32]:
toronto_groups.shape

(98, 334)

Let's find the Top 10 venues for each neighborhood and store that information in a new datafame.

In [33]:
# function to find and return most x most top venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_groups['Neighborhood']

for ind in np.arange(toronto_groups.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_groups.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Coffee Shop,Caribbean Restaurant,Sandwich Place,Lounge,Skating Rink,Clothing Store,Sushi Restaurant
1,"Alderwood, Long Branch",Discount Store,Convenience Store,Pizza Place,Pharmacy,Print Shop,Shopping Mall,Garden Center,Gas Station,Liquor Store,Donut Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Sandwich Place,Park,Supermarket,Frozen Yogurt Shop,Chinese Restaurant,Gas Station,Fried Chicken Joint,Shopping Mall
3,Bayview Village,Bank,Grocery Store,Japanese Restaurant,Gas Station,Chinese Restaurant,Park,Restaurant,Café,Skating Rink,Trail
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Park,Pizza Place,Sandwich Place,Bank,Thai Restaurant,Bakery,Comfort Food Restaurant,Skating Rink


##### Let's cluster the neighborhoods and visualize the data.

We are going to run *k*-means clustering to cluster the neighborhoods into 5 clusters.

In [35]:
# set number of clusters
kclusters = 5

toronto_clustering = toronto_groups.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([3, 3, 3, 3, 2, 2, 0, 2, 2, 2, 3, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 3,
       2, 3, 3, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2, 2, 3, 3, 0, 3, 2, 3, 3,
       2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 1, 3, 4, 2, 3, 3, 2, 2, 2,
       0, 2, 0, 2, 3, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 0,
       3, 3, 3, 2, 2, 3, 0, 2, 0, 4], dtype=int32)

Now we can make a new dataframe that has the cluster and the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_final = df_merge

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_final = toronto_final.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_final.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Bus Stop,Convenience Store,Shopping Mall,Pharmacy,Food & Drink Shop,Train Station,Fast Food Restaurant,Café,Supermarket
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Coffee Shop,Men's Store,Boxing Gym,Sporting Goods Shop,Portuguese Restaurant,Grocery Store,Hockey Arena,Golf Course,Intersection,Gym / Fitness Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Pub,Café,Park,Theater,Breakfast Spot,Bakery,Diner,Restaurant,Brewery
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Furniture / Home Store,Fried Chicken Joint,Vietnamese Restaurant,Dessert Shop,Sushi Restaurant,Sandwich Place
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Park,Pizza Place,Sushi Restaurant,Italian Restaurant,Ramen Restaurant,Café,Gay Bar,Bookstore,Clothing Store


Now, we can visualize the clusters on a map like we looked at before. The data will be clustered and labeled with the Neighborhood names and the Cluster labels. We will work on that next.


In [37]:
toronto_final['Cluster Labels'] = toronto_final['Cluster Labels'].fillna(0)
toronto_final['Cluster Labels'] = toronto_final['Cluster Labels'].astype(int)

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Neighbourhood'], toronto_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's explore the clusters and see if we can find some good labels for them.

We will look at each cluster and try to detrmine the commonalities between them.

**Cluster 1**

Cluster 1 appears to have alot of parks and playgrounds. We will call this one 'Recreation'.

In [39]:
toronto_final.loc[toronto_final['Cluster Labels'] == 0, toronto_final.columns[[1] + list(range(5, toronto_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,0,Park,Pizza Place,Hotel,Bank,Restaurant,Fish & Chips Shop,Clothing Store,Grocery Store,Mexican Restaurant,Gym
12,Scarborough,0,Playground,Burger Joint,Italian Restaurant,Park,Breakfast Spot,Food & Drink Shop,Field,Escape Room,Ethiopian Restaurant,Event Space
22,Scarborough,0,Coffee Shop,Park,Chinese Restaurant,Fast Food Restaurant,Indian Restaurant,Mobile Phone Shop,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
57,North York,0,Auto Workshop,Park,Golf Course,Convenience Store,Bakery,Intersection,Storage Facility,Discount Store,Gas Station,Zoo
58,Scarborough,0,Park,Convenience Store,Diner,Thai Restaurant,General Entertainment,Gym,Gym Pool,Skating Rink,Auto Workshop,College Stadium
66,North York,0,Park,Restaurant,Coffee Shop,Gym,Pet Store,Dog Run,Chinese Restaurant,Bowling Alley,Playground,Grocery Store
91,Downtown Toronto,0,Coffee Shop,Park,Grocery Store,Japanese Restaurant,BBQ Joint,Filipino Restaurant,Juice Bar,Bank,Athletics & Sports,Candy Store
95,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Cluster 2**

Cluster 2 only contains one neighborhood, and it has a Zoo and Lounges in it. We are going to call it 'Zoo/Entertainment'

In [40]:
toronto_final.loc[toronto_final['Cluster Labels'] == 1, toronto_final.columns[[1] + list(range(5, toronto_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,1,Lounge,Coffee Shop,Zoo,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


**Cluster 3**

Cluster 3 is a large cluster with many coffee shops (Tim Horton's probably) and restraunts. We will call this one 'Dining'.

In [41]:
toronto_final.loc[toronto_final['Cluster Labels'] == 2, toronto_final.columns[[1] + list(range(5, toronto_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Coffee Shop,Pub,Café,Park,Theater,Breakfast Spot,Bakery,Diner,Restaurant,Brewery
3,North York,2,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Furniture / Home Store,Fried Chicken Joint,Vietnamese Restaurant,Dessert Shop,Sushi Restaurant,Sandwich Place
4,Downtown Toronto,2,Coffee Shop,Park,Pizza Place,Sushi Restaurant,Italian Restaurant,Ramen Restaurant,Café,Gay Bar,Bookstore,Clothing Store
6,Scarborough,2,Fast Food Restaurant,Coffee Shop,Trail,Bank,Paper / Office Supplies Store,Bakery,Park,Gym,Caribbean Restaurant,Supermarket
7,North York,2,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Burger Joint,Bank,Supermarket,Beer Store,Pizza Place,Asian Restaurant
9,Downtown Toronto,2,Coffee Shop,Gastropub,Italian Restaurant,Japanese Restaurant,Hotel,Diner,Creperie,Restaurant,New American Restaurant,Sushi Restaurant
13,North York,2,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Burger Joint,Bank,Supermarket,Beer Store,Pizza Place,Asian Restaurant
14,East York,2,Park,Coffee Shop,Café,Skating Rink,Sandwich Place,Pizza Place,Curling Ice,Beer Store,Thai Restaurant,Pastry Shop
15,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Beer Bar,Hotel,Gastropub,Bakery,Italian Restaurant
19,East Toronto,2,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Burger Joint,Caribbean Restaurant,Bar,Sandwich Place


**Cluster 4**

Cluster 4 looks like it has a lot of shopping and dining in it. Probably Mostly residential based on the tupes of places we are seeing. We will go with 'Residential'.

In [42]:
toronto_final.loc[toronto_final['Cluster Labels'] == 3, toronto_final.columns[[1] + list(range(5, toronto_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Park,Bus Stop,Convenience Store,Shopping Mall,Pharmacy,Food & Drink Shop,Train Station,Fast Food Restaurant,Café,Supermarket
1,North York,3,Coffee Shop,Men's Store,Boxing Gym,Sporting Goods Shop,Portuguese Restaurant,Grocery Store,Hockey Arena,Golf Course,Intersection,Gym / Fitness Center
5,Etobicoke,3,Pharmacy,Bakery,Bank,Shopping Mall,Convenience Store,Grocery Store,Park,Playground,Café,Skating Rink
8,East York,3,Gym / Fitness Center,Brewery,Construction & Landscaping,Pizza Place,Coffee Shop,Rock Climbing Spot,Gastropub,Bank,Bakery,Fast Food Restaurant
10,North York,3,Grocery Store,Fast Food Restaurant,Gas Station,Coffee Shop,Pizza Place,Italian Restaurant,Pub,Furniture / Home Store,Park,Latin American Restaurant
16,York,3,Pizza Place,Coffee Shop,Convenience Store,Optical Shop,Bagel Shop,Korean Restaurant,Bank,Sandwich Place,Gastropub,Dance Studio
17,Etobicoke,3,Coffee Shop,Shopping Mall,College Rec Center,Gas Station,Grocery Store,Farmers Market,Pizza Place,Beer Store,Intersection,Café
18,Scarborough,3,Pizza Place,Fast Food Restaurant,Bank,Coffee Shop,Pharmacy,Liquor Store,Bus Line,Sandwich Place,Supermarket,Greek Restaurant
21,York,3,Pharmacy,Park,Pizza Place,Portuguese Restaurant,Food Truck,Bus Stop,Mexican Restaurant,Fast Food Restaurant,Falafel Restaurant,Bakery
26,Scarborough,3,Coffee Shop,Pharmacy,Gas Station,Indian Restaurant,Bank,Bakery,Asian Restaurant,Chinese Restaurant,Grocery Store,Thai Restaurant


**Cluster 5**

Cluster 5 is an odd ball. We have Parks again, but also a pool and Elementary Schools. We will call it 'Other' for lack of a better definition.

In [43]:
toronto_final.loc[toronto_final['Cluster Labels'] == 4, toronto_final.columns[[1] + list(range(5, toronto_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,4,Park,Pool,Field,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
101,Etobicoke,4,Park,Ice Cream Shop,Italian Restaurant,Bus Stop,Shopping Mall,Gym / Fitness Center,Eastern European Restaurant,Design Studio,Department Store,Ethiopian Restaurant


Now that we have defined our clusters, let's build one last map with the labels included. The code will look at the cluster label value and assign a Text value to the map popup based on it.

In [46]:
labels = ['Recreation', 'Zoo/Entertainment', 'Dining', 'Residential', 'Other']

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Neighbourhood'], toronto_final['Cluster Labels']):
    label = folium.Popup(labels[cluster], parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<hr>

**Thank you for viewing this notebook!**

This notebook was created by [Tony Allen Price](http://www.tonyallenprice.com) as a part of a course on **Coursera** called *Applied Data Science Capstone*. [Click here](https://www.coursera.org/learn/applied-data-science-capstone) to learn more about the course.

All content, including source code, is released under  terms of the the [MIT License](https://bigdatauniversity.com/mit-license?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).